In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from model import ResNet50, ResNet38, ResNet26

# 논문 


In [2]:

## 데이터 셋은 CIFAR10으로
num_classes = 10
batch_size = 32
num_workers = 4

## 데이터셋 로드
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616)
    )
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616)
    )
])
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=True, download=True, transform=transform_train),
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, transform=transform_test),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)

Files already downloaded and verified


In [3]:
stem = False
model = ResNet26(num_classes=num_classes, stem=stem)
model = model.cuda()

In [4]:
start_epoch = 1
best_acc = 0.0

lr = 0.01
momentum =0.9
weight_decay = 0.0001


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)


In [5]:
epochs = 100
print_interval = 20

## 30에폭 마다 러닝레이트 줄임
def adjust_learning_rate(optimizer, epoch):
    decayed_lr = lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = decayed_lr

for epoch in range(1,epochs+1):
    model.train()
    train_acc = 0.0
    step = 0
    for data, target in train_loader:
        adjust_learning_rate(optimizer, epoch)
        data, target = data.cuda(), target.cuda()
    
    
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


        ## onnx 저장


        y_pred = output.data.max(1)[1]
        
        acc = float(y_pred.eq(target.data).sum()) / len(data) * 100.
        train_acc += acc
        step += 1
        if step % print_interval == 0:
            print("Epoch ",epoch," ",len(train_loader)," / ",step,"batch Loss: ",loss.item(),"  ACC: ",acc,"%")
    
    
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            prediction = output.data.max(1)[1]
            correct += prediction.eq(target.data).sum()

    acc = 100. * float(correct) / len(test_loader.dataset)
    print('Test acc: {0:.2f} \n'.format(acc))

Epoch  1   1563  /  20 batch Loss:  2.245594024658203   ACC:  31.25 %
Epoch  1   1563  /  40 batch Loss:  2.626516103744507   ACC:  18.75 %
Epoch  1   1563  /  60 batch Loss:  2.2531518936157227   ACC:  15.625 %
Epoch  1   1563  /  80 batch Loss:  2.061490297317505   ACC:  31.25 %
Epoch  1   1563  /  100 batch Loss:  2.0909833908081055   ACC:  31.25 %
Epoch  1   1563  /  120 batch Loss:  2.115436553955078   ACC:  28.125 %
Epoch  1   1563  /  140 batch Loss:  2.0981976985931396   ACC:  12.5 %
Epoch  1   1563  /  160 batch Loss:  2.080796957015991   ACC:  12.5 %
Epoch  1   1563  /  180 batch Loss:  1.9663963317871094   ACC:  31.25 %
Epoch  1   1563  /  200 batch Loss:  1.8342472314834595   ACC:  37.5 %
Epoch  1   1563  /  220 batch Loss:  1.7848618030548096   ACC:  40.625 %
Epoch  1   1563  /  240 batch Loss:  1.78364896774292   ACC:  28.125 %
Epoch  1   1563  /  260 batch Loss:  2.096372604370117   ACC:  28.125 %
Epoch  1   1563  /  280 batch Loss:  2.094644784927368   ACC:  28.125 %
Ep

KeyboardInterrupt: 

In [ ]:
## onnx 저장
## operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK 꼭 해줘야함
## unfold 연산을 
# https://github.com/pytorch/pytorch/issues/14395#issuecomment-444697403


# import torch.onnx
# torch.onnx.export(model.cpu(),data.cpu(),'./stand_alone_self.onnx',export_params=False,opset_version=12,
#                   operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

In [ ]:
## 모델 저장
torch.save(model.state_dict(),"./temp_save_model/ch.pth")

## 모델 불러오기
model.load_state_dict(torch.load("./temp_save_model/ch.pth"))

In [ ]:
model

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


## 모델에 넣을데이터 출력
img = data[7:8,:,:,:]
print("인풋 이미지 텐서 사이즈",img.shape)
print("이게뭐지?(타겟값)",target[7:8])
img_test = img.detach().cpu().numpy()
img_test = img_test.transpose(0,2,3,1)
img_test = img_test[0,:,:,:]
plt.axis("off");
plt.imshow(img_test);
plt.show();

In [ ]:
model_children = list(model.children())
print("모델의 모듈 구조길이 : ",len(model_children))

In [ ]:
# for i,module in enumerate(model_children):
#     print(i,"번째 모듈 \n",module,"\n")
model_children[0]

In [ ]:
results = [model_children[0](img)]

print("인풋 이미지 데이터가 첫번째 컨볼루션 레이어(스탬) 통과후 텐서 쉐이프 = ",results[0].shape)


In [ ]:
## 인풋 데이터 모든 레이어 통과를 result 리스트에 저장
for i in range(1, len(model_children)-1):
    # 마지막 레이어의 결과를 다음 레이어로 전달
    results.append(model_children[i](results[-1]))
outputs = results

In [ ]:
print("1번째 레이어 생김새","\n",model_children[1],"\n")
print("통과후 텐서 쉐이프",outputs[1].shape)

In [ ]:
# 1. cnn 통과후  torch.Size([32, 64, 32, 32])
# 2. 어텐션 통과후  torch.Size([32, 64, 32, 32])
# 3.cnn 통과후  torch.Size([32, 256, 32, 32])
# 숏컷 연산후 torch.Size([32, 256, 32, 32])